In [ ]:
%%bash
pip install mxnet keras keras-mxnet matplotlib opencv-python sklearn

# Preparation

In [1]:
import mxnet as mx
import numpy as np
import keras as k
from matplotlib.pyplot import imshow
import cv2

%matplotlib inline

Using MXNet backend


In [2]:
SHAPE=(224, 224, 3)
CLASSES=4

import helpers.labelinputgenerator
trainData = helpers.labelinputgenerator.LabelInputGenerator("train", SHAPE, CLASSES)
testData = helpers.labelinputgenerator.LabelInputGenerator("train", SHAPE, CLASSES, shuffle=False)

print("Number of batches in the training data: ", len(trainData))
print("Number of batches in the Validation data: ", len(trainData))

Number of batches in the training data:  5
Number of batches in the Validation data:  5


# Test image

In [ ]:
for x, y in trainData:
    i = 3
    imshow(x[i])
    print(y)
    break

# Setup NN

In [3]:
import models.simplemobilenet as m

NNmodel = m.create_model(input_shape=SHAPE, outputs=3)
NNmodel.summary()

/Users/SEIDLM/workspaces/privat/ubiquitous-adventure/venv/lib/python3.7/site-packages/keras/backend/mxnet_backend.py:94: UserWarning: MXNet Backend performs best with `channels_first` format. Using `channels_last` will significantly reduce performance due to the Transpose operations. For performance improvement, please use this API`keras.utils.to_channels_first(x_input)`to transform `channels_last` data to `channels_first` format and also please change the `image_data_format` in `keras.json` to `channels_first`.Note: `x_input` is a Numpy tensor or a list of Numpy tensorRefer to: https://github.com/awslabs/keras-apache-mxnet/tree/master/docs/mxnet_backend/performance_guide.md
  train_symbol = func(*args, **kwargs)
/Users/SEIDLM/workspaces/privat/ubiquitous-adventure/venv/lib/python3.7/site-packages/keras/backend/mxnet_backend.py:97: UserWarning: MXNet Backend performs best with `channels_first` format. Using `channels_last` will significantly reduce performance due to the Transpose oper

ValueError: Output tensors to a Model must be the output of a Keras `Layer` (thus holding past layer metadata). Found: <keras.layers.core.Dense object at 0x1416d9590>

In [ ]:
NUM_EPOCHS = 1
opt = k.optimizers.Adam()
NNmodel.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
history = NNmodel.fit_generator(
	trainData,
	validation_data=testData,
	epochs=NUM_EPOCHS)

## Save model

In [ ]:
# serialize model to JSON
model_json = NNmodel.to_json()
with open("result/classification_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
NNmodel.save_weights("result/classification_model.h5")
print("Saved model to disk")

# Evaluate Result

In [ ]:
valData = helpers.labelinputgenerator.LabelInputGenerator("train", SHAPE, 3, shuffle=False)
evaluation = NNmodel.evaluate_generator(valData)


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

accuracy = history.history['acc']
val_accuracy = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))

plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

print('Validation loss:', evaluation[0])
print('Validation accuracy:', evaluation[1])

# Manual Evaluation

In [ ]:
prediction = NNmodel.predict_generator(valData)
prediction_classes = np.argmax(prediction, axis=1)

In [ ]:
val_y = valData.all_labels()

In [ ]:
from sklearn.metrics import classification_report
target_names = ["None", "Cat", "Dog", "Human"]
print(classification_report(val_y, prediction_classes, target_names=target_names))